In [1]:
# Imports
import numpy as np
import pandas as pd
import os
from PIL import Image
from tensorflow.keras.utils import load_img
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model

from tqdm.notebook import tqdm_notebook

2022-12-27 22:23:20.804243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 22:23:21.550746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-27 22:23:21.550800: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-27 22:23:21.550806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

> I'll eventually get around to dealing with these error messages.
>
> Apparently, these are *warnings* not errors.

> The following code is adapted from Gabe Flomo's [tutorial](https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34) on clustering images by similarity.


In [2]:
# Construct the model
model = VGG16()
# Remove the output layer
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
model

2022-12-27 22:23:22.253091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-27 22:23:22.253124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michael-XPS-8700): /proc/driver/nvidia/version does not exist
2022-12-27 22:23:22.253355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 22:23:22.524379: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-12-27 22:23:22.647156: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of fre

In [3]:
# Load and resize the images. The columns of the image_data list is vessel_ID
# followed by image_ID
vessel_ids = []
image_ids = []
images = []
feature_vectors = []
for filename in tqdm_notebook(os.listdir('../images/')[0:2000]):
    try:
        # Open the image, resize it, and convert it to a numpy array
        image = load_img('../images/' + filename)
        image.thumbnail((224, 224))
        image = np.asarray(image)
        
        # Reshape the image for use with the VGG16 model
        image = image.reshape(1, 224, 224, 3)
        # Run the image through the preprocessor
        image = preprocess_input(image)
        # Construct a feature vector using vgg16
        feature = model.predict(image)
        
        # Get the vessel id and image id from the filename
        temp = filename.split('_')
        
        # Store the images, features, and ids in lists
        vessel_ids.append(temp[1])
        image_ids.append(temp[2][:-4])
        images.append(image)
        feature_vectors.append(image)
    except:
        # Keep track of the files we couldn't open.
        print('Could not open', filename)
        #with open('bad_files.txt', 'a') as f:
        #    f.write(filename + '\n')

  0%|          | 0/2000 [00:00<?, ?it/s]

Could not open drawing_18688_15819.jpg
Could not open drawing_17496_13961.jpg
Could not open drawing_17334_13780.jpg
Could not open drawing_19762_18235.jpg
Could not open drawing_28755_40646.jpg
Could not open drawing_22232_22841.jpg
Could not open drawing_15947_10852.jpg
Could not open drawing_27761_39680.jpg
Could not open drawing_19300_17680.jpg
Could not open drawing_20289_16451.jpg
Could not open drawing_21240_20500.jpg
Could not open drawing_23361_25661.jpg
Could not open drawing_24493_28098.jpg
Could not open drawing_26734_33815.jpg
Could not open drawing_24044_26905.jpg
Could not open drawing_24584_28576.jpg
Could not open drawing_21639_21074.jpg
Could not open drawing_20340_16572.jpg
Could not open drawing_25975_33175.jpg
Could not open drawing_18611_15742.jpg
Could not open drawing_19230_16764.jpg
Could not open drawing_22604_23981.jpg
Could not open drawing_17641_14203.jpg
Could not open drawing_20098_18572.jpg
Could not open drawing_19018_16167.jpg
Could not open drawing_18

Could not open drawing_18851_15992.jpg
Could not open drawing_24562_28458.jpg
Could not open drawing_27200_35127.jpg
Could not open drawing_20286_16447.jpg
Could not open drawing_27009_34179.jpg
Could not open drawing_20207_18688.jpg
Could not open drawing_15040_9091.jpg
Could not open drawing_20243_18724.jpg
Could not open drawing_22105_22442.jpg
Could not open drawing_18890_16031.jpg
Could not open drawing_26971_34217.jpg
Could not open drawing_25129_30118.jpg
Could not open drawing_19364_17746.jpg
Could not open drawing_17730_14333.jpg
Could not open drawing_26568_33685.jpg
Could not open drawing_17708_14313.jpg
Could not open drawing_16900_12822.jpg
Could not open drawing_23047_25097.jpg
Could not open drawing_21789_21383.jpg
Could not open drawing_26574_33799.jpg
Could not open drawing_24555_28409.jpg
Could not open drawing_437_526.jpg
Could not open drawing_464_579.jpg
Could not open drawing_22826_24804.jpg
Could not open drawing_27095_34815.jpg
Could not open drawing_24172_27191

Could not open drawing_24046_26907.jpg
Could not open drawing_27106_34583.jpg
Could not open drawing_18481_15602.jpg
Could not open drawing_18512_15635.jpg
Could not open drawing_17709_14314.jpg
Could not open drawing_20089_18563.jpg
Could not open drawing_18596_15725.jpg
Could not open drawing_19203_16694.jpg
Could not open drawing_22239_22863.jpg
Could not open drawing_28599_40174.jpg
Could not open drawing_18745_15877.jpg
Could not open drawing_26591_33893.jpg
Could not open drawing_28388_39395.jpg
Could not open drawing_17635_14197.jpg
Could not open drawing_20771_19812.jpg
Could not open drawing_28591_40165.jpg
Could not open drawing_23138_25252.jpg
Could not open drawing_25991_33156.jpg
Could not open drawing_20781_19828.jpg
Could not open drawing_26621_33863.jpg
Could not open drawing_15977_10909.jpg
Could not open drawing_26566_33683.jpg
Could not open drawing_12052_2944.jpg
Could not open drawing_28140_37742.jpg
Could not open drawing_26715_33838.jpg
Could not open drawing_231

Could not open drawing_17982_14713.jpg
Could not open drawing_22063_22320.jpg
Could not open drawing_1217_9972.jpg
Could not open drawing_662_968.jpg
Could not open drawing_16268_11522.jpg
Could not open drawing_28253_38639.jpg
Could not open drawing_26174_33384.jpg
Could not open drawing_26669_33728.jpg
Could not open drawing_19120_16343.jpg
Could not open drawing_28177_38057.jpg
Could not open drawing_23535_25984.jpg
Could not open drawing_551_13646.jpg
Could not open drawing_24542_28424.jpg
Could not open drawing_20349_16590.jpg
Could not open drawing_26241_33317.jpg
Could not open drawing_27111_34565.jpg
Could not open drawing_22756_24727.jpg
Could not open drawing_22983_24994.jpg
Could not open drawing_17390_13842.jpg
Could not open drawing_24382_27715.jpg
Could not open drawing_24250_27405.jpg
Could not open drawing_18312_15418.jpg
Could not open drawing_20147_18621.jpg
Could not open drawing_17147_13212.jpg
Could not open drawing_22085_22375.jpg
Could not open drawing_22293_2310

Could not open drawing_22945_24957.jpg
Could not open drawing_16780_12641.jpg
Could not open drawing_19259_17639.jpg
Could not open drawing_14962_8843.jpg
Could not open drawing_22937_24948.jpg
Could not open drawing_16274_11527.jpg
Could not open drawing_18931_16074.jpg
Could not open drawing_16092_11214.jpg
Could not open drawing_20125_18599.jpg
Could not open drawing_17342_13789.jpg
Could not open drawing_21074_20336.jpg
Could not open drawing_17232_13387.jpg
Could not open drawing_27299_35315.jpg
Could not open drawing_18600_15729.jpg
Could not open drawing_22542_23616.jpg
Could not open drawing_18843_15984.jpg
Could not open drawing_26636_33761.jpg
Could not open drawing_26028_33118.jpg
Could not open drawing_20092_18566.jpg
Could not open drawing_25918_32866.jpg
Could not open drawing_26234_33324.jpg
Could not open drawing_26762_33782.jpg
Could not open drawing_16606_12312.jpg
Could not open drawing_26205_33353.jpg
Could not open drawing_19138_16380.jpg
Could not open drawing_161

Could not open drawing_17820_14451.jpg
Could not open drawing_19905_18357.jpg
Could not open drawing_12297_3497.jpg
Could not open drawing_22574_23951.jpg
Could not open drawing_16318_11584.jpg
Could not open drawing_26881_34041.jpg
Could not open drawing_26783_33986.jpg
Could not open drawing_28777_40657.jpg
Could not open drawing_26573_33788.jpg
Could not open drawing_22984_24995.jpg
Could not open drawing_140_152.jpg
Could not open drawing_24069_26944.jpg
Could not open drawing_26419_33511.jpg
Could not open drawing_19429_17838.jpg
Could not open drawing_15180_9448.jpg
Could not open drawing_26313_33244.jpg
Could not open drawing_23227_25420.jpg
Could not open drawing_27189_35115.jpg
Could not open drawing_24081_38545.jpg
Could not open drawing_19155_16397.jpg
Could not open drawing_18364_15475.jpg
Could not open drawing_20551_19442.jpg
Could not open drawing_15877_10776.jpg
Could not open drawing_27314_35337.jpg
Could not open drawing_25430_31436.jpg
Could not open drawing_16170_11

Could not open drawing_23678_26193.jpg
Could not open drawing_26100_33046.jpg
Could not open drawing_16543_12202.jpg
Could not open drawing_27102_34579.jpg
Could not open drawing_19754_18227.jpg
Could not open drawing_16080_11197.jpg
Could not open drawing_19163_16652.jpg
Could not open drawing_26647_33750.jpg
Could not open drawing_25493_31626.jpg
Could not open drawing_23184_25329.jpg
Could not open drawing_20056_18530.jpg
Could not open drawing_22501_23467.jpg
Could not open drawing_22911_24922.jpg
Could not open drawing_24180_27216.jpg
Could not open drawing_21355_20621.jpg
Could not open drawing_16820_33447.jpg
Could not open drawing_18829_15970.jpg
Could not open drawing_17536_14030.jpg
Could not open drawing_20287_16449.jpg
Could not open drawing_18039_16178.jpg
Could not open drawing_26697_33856.jpg
Could not open drawing_23179_25318.jpg
Could not open drawing_24458_27833.jpg
Could not open drawing_92_14808.jpg
Could not open drawing_18707_15838.jpg
Could not open drawing_12254

Could not open drawing_15981_10913.jpg
Could not open drawing_24141_27120.jpg
Could not open drawing_19569_17983.jpg
Could not open drawing_16642_12359.jpg
Could not open drawing_12343_3590.jpg
Could not open drawing_24164_27171.jpg
Could not open drawing_840_9984.jpg
Could not open drawing_465_581.jpg
Could not open drawing_22682_24158.jpg
Could not open drawing_26736_33813.jpg
Could not open drawing_24548_28475.jpg
Could not open drawing_12162_3167.jpg
Could not open drawing_24276_27484.jpg
Could not open drawing_18900_16041.jpg
Could not open drawing_19255_17638.jpg
Could not open drawing_27192_35112.jpg
Could not open drawing_24565_28480.jpg
Could not open drawing_16251_11486.jpg
Could not open drawing_16922_12844.jpg
Could not open drawing_24408_27760.jpg
Could not open drawing_371_448.jpg
Could not open drawing_18886_16026.jpg
Could not open drawing_26451_33481.jpg
Could not open drawing_1241_9966.jpg
Could not open drawing_19685_18158.jpg
Could not open drawing_23068_25124.jpg
C

Could not open drawing_28643_40219.jpg
Could not open drawing_18878_16019.jpg
Could not open drawing_27325_35351.jpg
Could not open drawing_18352_15463.jpg
Could not open drawing_240_278.jpg
Could not open drawing_26820_34017.jpg
Could not open drawing_24738_28975.jpg
Could not open drawing_22503_23471.jpg
Could not open drawing_27908_38951.jpg
Could not open drawing_23285_25533.jpg
Could not open drawing_19534_17937.jpg
Could not open drawing_1088_10703.jpg
Could not open drawing_26255_33303.jpg
Could not open drawing_24412_27769.jpg
Could not open drawing_27023_34117.jpg
Could not open drawing_23935_26738.jpg
Could not open drawing_19915_18367.jpg
Could not open drawing_17903_14611.jpg
Could not open drawing_26600_33884.jpg
Could not open drawing_27016_34171.jpg
Could not open drawing_12181_3215.jpg
Could not open drawing_12300_3505.jpg
Could not open drawing_23150_25273.jpg
Could not open drawing_27067_34275.jpg
Could not open drawing_25874_32937.jpg
Could not open drawing_347_425.j

> Got all of the images resized and in a list. 


In [8]:
# Store the ids and images in a dataframe
images_df = pd.DataFrame(list(zip(vessel_ids, 
                                  image_ids, 
                                  images, 
                                  feature_vectors)),
                         columns = ['vessel_ID',
                                    'image_ID',
                                    'image',
                                    'feature_vectors'])
images = images_df['image'].copy()
images

0    [[[[151.061   138.22101 131.32   ], [151.061  ...
1    [[[[151.061   138.22101 131.32   ], [151.061  ...
2    [[[[151.061   138.22101 131.32   ], [151.061  ...
Name: image, dtype: object

In [5]:
# Reduce the dimension of the feature vector using PCA